In [ ]:
!pip install conllu
!pip install transformers datasets seqeval
!pip install hf_xet
!pip install pyvi
import os

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=7b730c158d4f6f3087d671d0581c39de706a8d3bec75198d4f922cf9c3ae8512
  Stored in directory: /root/.cache/pip/wheels/bc/92/f0/243288f899c2eacdfa8c5f9aede4c71a9bad0ee26a01dc5ead
Successfully built seqeval
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 77.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.1 MB/s eta 0:00:00


In [ ]:
%%capture
!git clone https://github.com/VinAIResearch/PhoNER_COVID19.git
!git clone https://github.com/J4Fsake/NLP_NER_BERT.git

**Thực hiện analyze từng câu để biết được có bao nhiêu câu và số câu có độ dài lớn hơn max length (128).**

In [ ]:
import sys
sys.path.append('/content/NLP_NER_BERT')

import MyUtils

print('File train_word.conll')
MyUtils.analyze_sentence_lengths('/content/PhoNER_COVID19/data/word/train_word.conll')
print('--------------------')
print('File dev_word.conll')
MyUtils.analyze_sentence_lengths('/content/PhoNER_COVID19/data/word/dev_word.conll')
print('--------------------')
print('File test_word.conll')
MyUtils.analyze_sentence_lengths('/content/PhoNER_COVID19/data/word/test_word.conll')

File train_word.conll
Tổng số câu: 5027
Độ dài 161 có 1 câu
--------------------
File dev_word.conll
Tổng số câu: 2000
Độ dài 138 có 1 câu
--------------------
File test_word.conll
Tổng số câu: 3000
Độ dài 138 có 1 câu
Độ dài 139 có 1 câu
Độ dài 162 có 1 câu


**Đọc và xử lý từng câu trong file, đối với những câu có độ dài lớn hơn max length (128) thì sẽ thực hiện chèn khoảng trắng ("\n") để chia đôi câu đó ra.**

In [ ]:
#Preprocessing
!python /content/NLP_NER_BERT/preprocess.py /content/PhoNER_COVID19/data/word/train_word.conll vinai/phobert-large 128 > train.txt
!python /content/NLP_NER_BERT/preprocess.py /content/PhoNER_COVID19/data/word/dev_word.conll vinai/phobert-large 128 > dev.txt
!python /content/NLP_NER_BERT/preprocess.py /content/PhoNER_COVID19/data/word/test_word.conll vinai/phobert-large 128 > test.txt
!cat train.txt dev.txt test.txt | cut -d " " -f 2 | grep -v "^$"| sort | uniq > labels.txt

config.json: 100% 558/558 [00:00<00:00, 3.86MB/s]
vocab.txt: 100% 895k/895k [00:00<00:00, 4.73MB/s]
bpe.codes: 100% 1.14M/1.14M [00:00<00:00, 57.4MB/s]
tokenizer.json: 100% 3.13M/3.13M [00:00<00:00, 12.1MB/s]


**Thực hiện analyze lại các file đã được preprocess, ta sẽ thấy được các câu có độ dài lớn hơn max length (128) đã bị cắt ra.**

In [ ]:
MyUtils.analyze_sentence_lengths('train.txt')
MyUtils.analyze_sentence_lengths('dev.txt')
MyUtils.analyze_sentence_lengths('test.txt')

Tổng số câu: 5029
Tổng số câu: 2003
Tổng số câu: 3008


**Fine-tuning PhoBERT và sau load model pretrained và bắt đầu train.**

In [ ]:
# Model Arguments
BERT_MODEL = "vinai/phobert-large"
CONFIG_NAME = ""
TASK_TYPE = "NER"
TOKENIZER_NAME = ""
USE_FAST = False
CACHE_DIR = ""

# Data Training Arguments
DATA_DIR = "/content"
LABELS = "/content/labels.txt"
MAX_LENGTH = 128
OVERWRITE_CACHE = ""

# Training Arguments
OUTPUT_DIR = "/content/pho_bert_large/lr5e-5_bs16"
BATCH_SIZE = 16
LEARNING_RATE = 5e-5
WEIGHT_DECAY = 0.0
NUM_EPOCHS = 1
SAVE_STEPS = 750
SEED = 42
SAVE_TOTAL_LIMIT = 5

os.makedirs(OUTPUT_DIR, exist_ok=True)

!python /content/NLP_NER_BERT/run_ner.py \
  --model_name_or_path $BERT_MODEL \
  --task_type $TASK_TYPE \
  --data_dir $DATA_DIR \
  --labels $LABELS \
  --max_seq_length $MAX_LENGTH \
  --output_dir $OUTPUT_DIR \
  --overwrite_output_dir \
  --learning_rate $LEARNING_RATE \
  --weight_decay $WEIGHT_DECAY \
  --num_train_epochs $NUM_EPOCHS \
  --per_device_train_batch_size $BATCH_SIZE \
  --per_device_eval_batch_size $BATCH_SIZE \
  --save_total_limit $SAVE_TOTAL_LIMIT \
  --save_steps $SAVE_STEPS \
  --seed $SEED \
  --do_train \
  --do_eval \
  --do_predict

2025-06-05 05:51:23.583579: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749102683.616138    2072 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749102683.626308    2072 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-05 05:51:23.657143: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
INFO:__main__:Training/evaluation parameters TrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispat

In [ ]:
MyUtils.analyze_sentence_lengths('/content/pho_bert_large/lr5e-5_bs16/test_predictions.txt')

Tổng số câu: 3008
